In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-09-05 08:10:26


In [4]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2012,2017)
MaxOpenProcesses = 15
Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening'
ReceivedDir  = Project_DIR + '\\2.Received_Data'
Project_GDB  = Project_DIR + '\\5.NM_NS_GDB.gdb'

Src_Loc  = ReceivedDir + '\\crash_asof_11jun2018.csv'
Src_Unit = ReceivedDir + '\\vehicle_asof_11jun2018.csv'
Src_Occ  = ReceivedDir + '\\occupant_asof_11jun2018.csv'

HPMS105 = {year:ReceivedDir + '\\v105\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}
HPMS10  = {year:ReceivedDir + '\\v10\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}

Routes  = {year:Project_GDB + '\\Routes_{}'.format(2016) for year in Years}
AttTabs = {year:Project_GDB + '\\SegAtt_{}'.format(year) for year in Years}
Segs    = {year:Project_GDB + '\\Seg_{}'.format(year) for year in Years}
CrashData = {year:Project_GDB + '\\Crash_{}'.format(year) for year in Years}

UTM13N = arcpy.SpatialReference(26913)
NAD83_NM = arcpy.SpatialReference(6529)

RouteID = "ROUTE_ID"
BEG_STA = 'BEG_POINT'
END_STA = 'END_POINT'
Fields = ['F_SYSTEM','THROUGH_LANES','FACILITY_TYPE','AADT','MEDIAN_TYPE','SPEED_LIMIT','LANE_WIDTH']

BridgeUnder = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeUnder'
BridgeLRS   = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeLRS'
Signals     = ReceivedDir + '\\v10\\jg_request.gdb\\SignalizedIntersections'

2018-09-05 08:13:55


In [ ]:
arcpy.management.Delete(Project_GDB + '\\Int_Points')
hsmpy3.network.CON_CreateIntPoints(Project_DIR,
                                   HSMPY_PATH,
                                   Routes[2016],
                                   Project_GDB,
                                   r'C:\Users\mr068144\Downloads\NM\Bridge_Data.xlsx',
                                   r'C:\Users\mr068144\Downloads\NM\Signal_Data.xlsx',
                                   Project_GDB + '\\Int_Points',
                                   Project_DIR + '\\App_DF.csv',
                                   Title='')

In [5]:
Routes = hsmpy3.common.FCtoDF(Routes[2016],True)
Routes.index = pd.MultiIndex.from_tuples([(rid,bmp,emp) for rid,bmp,emp in zip(Routes.ROUTE_ID,Routes.BEG_POINT,Routes.END_POINT)])
Routes.shape

(212703, 7)

In [5]:
Segs = hsmpy3.common.FCtoDF(Segs[2016])
Segs.index = pd.MultiIndex.from_tuples([(rid,bmp,emp) for rid,bmp,emp in zip(Segs.ROUTE_ID,Segs.BEG_POINT,Segs.END_POINT)])

In [4]:
Int_DF = pd.read_csv(Project_DIR + '\\Int_DF.csv')
Int_DF['RIDList'] = [eval(s) for s in Int_DF['RIDList']]
Int_DF.shape

(102604, 6)

In [5]:
App_DF = pd.read_csv(Project_DIR + '\\App_DF.csv')
App_DF.shape

(357824, 18)

In [6]:
def List(L): return(';'.join([str(round(l)) for l in L]))
df = pd.DataFrame(App_DF.groupby(['ClusterNum','RID','MP'])['Deg'].aggregate(List))
df['ClusterNum'] = [i[0] for i in list(df.index)]
df['RID'] = [i[1] for i in list(df.index)]
df['MP' ] = [i[2] for i in list(df.index)]
df['AADT'] = [Segs[(Segs.ROUTE_ID==RID) & (Segs.BEG_POINT<=MP)  & (Segs.END_POINT>=MP)]['AADT']  for RID,MP in zip(df.RID,df.MP)]
df = df[df.ClusterNum!=0]
df

Deg  ClusterNum           RID  \
ClusterNum RID          MP                                                
1          BIA-AI0005-P 0.0000           17.0           1  BIA-AI0005-P   
                        0.0106     17.0;197.0           1  BIA-AI0005-P   
           LOC-236031-P 3.8190          195.0           1  LOC-236031-P   
           US-491-M     60.3628    99.0;279.0           1      US-491-M   
           US-491-P     60.3763   101.0;280.0           1      US-491-P   
2          BIA-AI009A-P 0.0000          359.0           2  BIA-AI009A-P   
                        0.0092    179.0;359.0           2  BIA-AI009A-P   
           US-491-M     15.1085    91.0;270.0           2      US-491-M   
           US-491-P     15.1753    90.0;271.0           2      US-491-P   
3          BIA-AIM010-P 6.3339    118.0;298.0           3  BIA-AIM010-P   
                        6.3477    122.0;302.0           3  BIA-AIM010-P   
           LOC-108988-P 0.1870          252.0           3  LOC-108988-P   
           LOC-108990-P 0.0000           35.0           3  LOC-108990-P   
5          BIA-IS0006-P 0.0000           78.0           5  BIA-IS0006-P   
                        0.0024     78.0;258.0           5  BIA-IS0006-P   
           BIA-IS0022-P 0.5337    178.0;261.0           5  BIA-IS0022-P   
           LOC-192535-P 0.0000          359.0           5  LOC-192535-P   
6          BIA-IS0010-P 1.7171    125.0;305.0           6  BIA-IS0010-P   
                        1.7307    124.0;305.0           6  BIA-IS0010-P   
           LOC-192600-P 0.4440          233.0           6  LOC-192600-P   
           LOC-192601-P 0.4130          226.0           6  LOC-192601-P   
7          BIA-NAV036-P 0.0000          351.0           7  BIA-NAV036-P   
                        0.0112    171.0;351.0           7  BIA-NAV036-P   
           US-491-M     87.9380    79.0;259.0           7      US-491-M   
           US-491-P     87.9389    79.0;259.0           7      US-491-P   
8          BL-11-M      4.6800     67.0;249.0           8       BL-11-M   
           BL-11-P      4.6710     69.0;249.0           8       BL-11-P   
           FL-1533-P    1.5884    160.0;340.0           8     FL-1533-P   
                        1.5980          340.0           8     FL-1533-P   
9          BL-11-M      3.7705     90.0;269.0           9       BL-11-M   
...                                       ...         ...           ...   
106125     LOC-293253-P 0.0000           62.0      106125  LOC-293253-P   
106126     LOC-112642-P 0.0040          187.0      106126  LOC-112642-P   
           LOC-112643-P 0.0680          272.0      106126  LOC-112643-P   
           LOC-112644-P 0.0040          187.0      106126  LOC-112644-P   
           LOC-112646-P 0.0000            1.0      106126  LOC-112646-P   
           LOC-112648-P 0.0000            1.0      106126  LOC-112648-P   
           LOC-112661-P 0.0000           91.0      106126  LOC-112661-P   
106127     LOC-131661-P 0.0270          230.0      106127  LOC-131661-P   
           LOC-131662-P 0.0270          230.0      106127  LOC-131662-P   
           LOC-131663-P 0.0410          254.0      106127  LOC-131663-P   
           LOC-131664-P 0.0410          254.0      106127  LOC-131664-P   
           LOC-131665-P 0.0000          320.0      106127  LOC-131665-P   
           LOC-131712-P 0.0000           72.0      106127  LOC-131712-P   
           LOC-131715-P 0.2420          146.0      106127  LOC-131715-P   
106131     LOC-264285-P 0.0060          266.0      106131  LOC-264285-P   
           LOC-264286-P 0.0000          346.0      106131  LOC-264286-P   
           LOC-264289-P 0.0000           63.0      106131  LOC-264289-P   
           LOC-264297-P 0.0730          145.0      106131  LOC-264297-P   
           LOC-264316-P 0.0000           84.0      106131  LOC-264316-P   
           US-180-M     0.0000           63.0      106131      US-180-M   
                        112.7074        205.0      106131      US-1

In [31]:
df['AADT2'] = [pd.DataFrame(x).max().item() for x in df.AADT]

In [9]:
df.to_csv('df09052018.csv')

In [ ]:
df.AADT = df.AADT.fillna(0)
df['Major'] = 0
intL = list(set(df.ClusterNum))
for intid in intL:
        Rdf = df[df.ClusterNum==intid]
        maxAADT = max(Rdf.AADT)
        i = list(Rdf[Rdf.AADT==maxAADT].index)[0]
        rid = Rdf.loc[i,'RID']
        ridL = [rid,hsmpy3.nm.OppositeDirID(rid)]
        for ind in list(Rdf.index):
            if Rdf.loc[ind,'RID'] in ridL:
                df.set_value(ind,'Major',1)
            else:
                df.set_value(ind,'Major',0)
df

In [8]:
df['ROUTE_ID'] = df.RID
Routes_DF = Routes
for i,r in df.iterrows():
        Rdf = Routes_DF[(Routes_DF['ROUTE_ID']==r.ROUTE_ID)]
        for pl in list(Rdf.Shape):
            for prt in pl:
                M = [s.M for s in prt]
                if M[0]<=r.MP and M[-1]>=r.MP:
                    df.set_value(i,'fpM',M[0])
                    df.set_value(i,'lpM',M[-1])
df

Deg  ClusterNum           RID  \
ClusterNum RID          MP                                                
1          BIA-AI0005-P 0.0000           17.0           1  BIA-AI0005-P   
                        0.0106     17.0;197.0           1  BIA-AI0005-P   
           LOC-236031-P 3.8190          195.0           1  LOC-236031-P   
           US-491-M     60.3628    99.0;279.0           1      US-491-M   
           US-491-P     60.3763   101.0;280.0           1      US-491-P   
2          BIA-AI009A-P 0.0000          359.0           2  BIA-AI009A-P   
                        0.0092    179.0;359.0           2  BIA-AI009A-P   
           US-491-M     15.1085    91.0;270.0           2      US-491-M   
           US-491-P     15.1753    90.0;271.0           2      US-491-P   
3          BIA-AIM010-P 6.3339    118.0;298.0           3  BIA-AIM010-P   
                        6.3477    122.0;302.0           3  BIA-AIM010-P   
           LOC-108988-P 0.1870          252.0           3  LOC-108988-P   
           LOC-108990-P 0.0000           35.0           3  LOC-108990-P   
5          BIA-IS0006-P 0.0000           78.0           5  BIA-IS0006-P   
                        0.0024     78.0;258.0           5  BIA-IS0006-P   
           BIA-IS0022-P 0.5337    178.0;261.0           5  BIA-IS0022-P   
           LOC-192535-P 0.0000          359.0           5  LOC-192535-P   
6          BIA-IS0010-P 1.7171    125.0;305.0           6  BIA-IS0010-P   
                        1.7307    124.0;305.0           6  BIA-IS0010-P   
           LOC-192600-P 0.4440          233.0           6  LOC-192600-P   
           LOC-192601-P 0.4130          226.0           6  LOC-192601-P   
7          BIA-NAV036-P 0.0000          351.0           7  BIA-NAV036-P   
                        0.0112    171.0;351.0           7  BIA-NAV036-P   
           US-491-M     87.9380    79.0;259.0           7      US-491-M   
           US-491-P     87.9389    79.0;259.0           7      US-491-P   
8          BL-11-M      4.6800     67.0;249.0           8       BL-11-M   
           BL-11-P      4.6710     69.0;249.0           8       BL-11-P   
           FL-1533-P    1.5884    160.0;340.0           8     FL-1533-P   
                        1.5980          340.0           8     FL-1533-P   
9          BL-11-M      3.7705     90.0;269.0           9       BL-11-M   
...                                       ...         ...           ...   
106125     LOC-293253-P 0.0000           62.0      106125  LOC-293253-P   
106126     LOC-112642-P 0.0040          187.0      106126  LOC-112642-P   
           LOC-112643-P 0.0680          272.0      106126  LOC-112643-P   
           LOC-112644-P 0.0040          187.0      106126  LOC-112644-P   
           LOC-112646-P 0.0000            1.0      106126  LOC-112646-P   
           LOC-112648-P 0.0000            1.0      106126  LOC-112648-P   
           LOC-112661-P 0.0000           91.0      106126  LOC-112661-P   
106127     LOC-131661-P 0.0270          230.0      106127  LOC-131661-P   
           LOC-131662-P 0.0270          230.0      106127  LOC-131662-P   
           LOC-131663-P 0.0410          254.0      106127  LOC-131663-P   
           LOC-131664-P 0.0410          254.0      106127  LOC-131664-P   
           LOC-131665-P 0.0000          320.0      106127  LOC-131665-P   
           LOC-131712-P 0.0000           72.0      106127  LOC-131712-P   
           LOC-131715-P 0.2420          146.0      106127  LOC-131715-P   
106131     LOC-264285-P 0.0060          266.0      106131  LOC-264285-P   
           LOC-264286-P 0.0000          346.0      106131  LOC-264286-P   
           LOC-264289-P 0.0000           63.0      106131  LOC-264289-P   
           LOC-264297-P 0.0730          145.0      106131  LOC-264297-P   
           LOC-264316-P 0.0000           84.0      106131  LOC-264316-P   
           US-180-M     0.0000           63.0      106131      US-180-M   
                        112.7074        205.0      106131      US-1

In [7]:
App_DF = df
App_DF['Int_ID2'] = df.ClusterNum
App_DF = App_DF[(App_DF.Int_ID2!=0) & (~pd.isnull(App_DF.MP))]

In [8]:
App_Tab = pd.DataFrame(columns=['RID','BMP','EMP','Int_ID2'])
x = 0
Routes_DF = Routes
for i,r in Routes_DF.iterrows():
        rid = r.ROUTE_ID
        app_all = App_DF[(App_DF.RID==rid)]
        mp_List = list(set(app_all.MP))
        mp_List.sort()
        int_id_List = [list(app_all[app_all.MP==mp]['Int_ID2'])[0] for mp in mp_List]
        for int_id,mp in zip(int_id_List,mp_List):
            app    = App_DF[(App_DF.RID==rid) & (App_DF.Int_ID2==int_id)]
            app_x  = App_DF[(App_DF.Int_ID2==int_id) & (App_DF.RID!=rid)]

            int_b = 250.0/5280.0

            bmp1 = max(mp - int_b,list(app.fpM)[0])
            emp3 = min(mp + int_b,list(app.lpM)[0])
            if emp3>bmp1:
                x += 1
                App_Tab.loc[x] = [rid,bmp1,emp3,int_id]

In [13]:
def NewBMPEMP(RID,BMP,EMP,NextRID,NextBMP,PrevRID,PrevEMP):
        NewBMP = BMP
        NewEMP = EMP
        if NextRID == RID:
            if NextBMP<EMP:
                NewEMP = np.mean([NextBMP,EMP])
        if PrevRID==RID:
            if PrevEMP>BMP:
                NewBMP = np.mean([PrevEMP,BMP])
        return([NewBMP,NewEMP])
if 1<2:
    App_Tab = App_Tab.sort_values(by=['RID','BMP'])
    r = list(App_Tab.RID)[1:]
    r.append('')
    App_Tab['NextRID'] = r
    r = list(App_Tab.BMP)[1:]
    r.append('')
    App_Tab['NextBMP'] = r

    r = ['']
    r.extend(list(App_Tab.RID)[:-1])
    App_Tab['PrevRID'] = r
    r = ['']
    r.extend(list(App_Tab.EMP)[:-1])
    App_Tab['PrevEMP'] = r

    App_Tab['NewBMPEMP']  = [NewBMPEMP(rid,bmp,emp,nrid,nbmp,prid,pemp) for rid,bmp,emp,nrid,nbmp,prid,pemp in 
                             zip(App_Tab.RID,App_Tab.BMP,App_Tab.EMP,App_Tab.NextRID,App_Tab.NextBMP,App_Tab.PrevRID,App_Tab.PrevEMP)]


In [16]:
if 1<2:
    App_Table = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt','AppTab','')
    arcpy.management.CreateTable(out_name=os.path.basename(App_Table),out_path=Project_GDB)
    arcpy.management.AddField(App_Table,'RID','Text')
    arcpy.management.AddField(App_Table,'BMP','Double')
    arcpy.management.AddField(App_Table,'EMP','Double')
    arcpy.management.AddField(App_Table,'Int_ID','Long')
    #arcpy.management.AddField(App_Table,'IntProx','Short')
    #arcpy.management.AddField(App_Table,'Major','Short')
    ic = arcpy.InsertCursor(App_Table)
    for i,r in App_Tab.iterrows():
        row = ic.newRow()
        row.setValue('RID',r.RID)
        row.setValue('BMP',r.NewBMPEMP[0])
        row.setValue('EMP',r.NewBMPEMP[1])
        row.setValue('Int_ID',r.Int_ID2)
        #row.setValue('Major',r.Major)
        #row.setValue('IntProx',r.IntProx)
        ic.insertRow(row)
    del ic
    del row    
    #Dissolve for Int_ID

In [43]:
for year in range(2012,2017):
    IntAppOverlay = hsmpy3.common.CreateOutPath(MainFile=Project_GDB+'\\Int_App_Att',appendix=str(year),Extension='')
    arcpy.lr.OverlayRouteEvents(
            in_table                 = Project_GDB+'\\Int_App_Event',
            in_event_properties      = ' '.join(['RID','LINE','BMP','EMP']), 
            overlay_table            = Project_GDB+'\\SegAtt_' + str(year),
            overlay_event_properties = ' '.join(['ROUTE_ID','LINE','BEG_POINT','END_POINT']),
            overlay_type             = "INTERSECT", 
            out_table                = IntAppOverlay, 
            out_event_properties     = ' '.join(['RID','LINE','BMP','EMP']),
            zero_length_events       = "ZERO", 
            in_fields                = "FIELDS", 
            build_index              = "INDEX"
    )    
    print(strftime("%Y-%m-%d %H:%M:%S"))

    df = hsmpy3.common.FCtoDF(IntAppOverlay)
    df['Major'] = 0
    intL = list(set(df.Int_ID))
    for intid in intL:
        Rdf = df[df.Int_ID==intid]
        maxAADT = max(Rdf.AADT)
        i = list(Rdf[Rdf.AADT==maxAADT].index)[0]
        rid = Rdf.loc[i,'RID']
        ridL = [rid,hsmpy3.nm.OppositeDirID(rid)]
        for ind in list(Rdf.index):
            if Rdf.loc[ind,'RID'] in ridL:
                df.set_value(ind,'Major',1)
            else:
                df.set_value(ind,'Major',0)
    print(strftime("%Y-%m-%d %H:%M:%S"))
    
    Int = hsmpy3.common.CreateOutPath(MainFile=Project_GDB+'\\Int',Extension='',appendix=str(year))
    arcpy.conversion.FeatureClassToFeatureClass(in_features=Project_GDB+'\\Int_Points',out_name=os.path.basename(Int),out_path=Project_GDB)
    arcpy.management.AddField(Int,'IFType','Text',10)
    arcpy.management.AddField(Int,'I_URBANCODE','Long')
    arcpy.management.AddField(Int,'FType_Major','Text',10)
    arcpy.management.AddField(Int,'FType_Minor','Text',10)
    arcpy.management.AddField(Int,'F_SYSTEM_Major','Short')
    arcpy.management.AddField(Int,'F_SYSTEM_Minor','Short')
    arcpy.management.AddField(Int,'OWNERSHIP_Major','Short')
    arcpy.management.AddField(Int,'OWNERSHIP_Minor','Short')
    arcpy.management.AddField(Int,'AADT_Major','Long')    
    arcpy.management.AddField(Int,'AADT_Minor','Long')    
    uc = arcpy.UpdateCursor(Int)
    for row in uc:
        int_id = row.getValue('Int_ID')
        signal = row.getValue('Signal')
        legs = row.getValue('Legs')
        type_major = ''
        
        idf = df[df.Int_ID==int_id]

        ft = list(idf[idf.Major==1]['AADT'])
        if len(ft)>0:
            row.setValue('AADT_Major',int(max(ft)))

        ft = list(idf[idf.Major==0]['AADT'])
        if len(ft)>0:
            row.setValue('AADT_Minor',int(max(ft)))

        ft = [t for t in set(list(idf[idf.Major==1]['FType'])) if t]
        if len(ft)>0:
            row.setValue('FType_Major',ft[0]) 
            type_major = ft[0]
        ft = [t for t in set(list(idf[idf.Major==0]['FType'])) if t]
        if len(ft)>0:
            row.setValue('FType_Minor',ft[0]) 

        ft = [t for t in set(list(idf[idf.Major==1]['F_SYSTEM'])) if t]
        if len(ft)>0:
            row.setValue('F_SYSTEM_Major',int(ft[0])) 

        ft = [t for t in set(list(idf[idf.Major==0]['F_SYSTEM'])) if t]
        if len(ft)>0:
            row.setValue('F_SYSTEM_Minor',int(ft[0])) 

        ft = [t for t in set(list(idf[idf.Major==1]['OWNERSHIP'])) if t]
        if len(ft)>0:
            row.setValue('OWNERSHIP_Major',int(ft[0]))

        ft = [t for t in set(list(idf[idf.Major==0]['OWNERSHIP'])) if t]
        if len(ft)>0:
            row.setValue('OWNERSHIP_Minor',int(ft[0]))

        ft = [t for t in set(list(idf[idf.Major==1]['URBAN_CODE'])) if t]
        if len(ft)>0:
            row.setValue('I_URBANCODE',int(ft[0])) 
        
        row.setValue('IFType',hsmpy3.nm.FindIntPG(Type_Major=type_major,Legs=legs,Signal=signal))
        
        uc.updateRow(row)   
    del uc
    del row
    print(strftime("%Y-%m-%d %H:%M:%S"))
    
    df1 = hsmpy3.common.FCtoDF(Int,selectedFields=['Int_ID','IFType'])
    df1.index = df1.Int_ID
    arcpy.management.AddField(IntAppOverlay,'IFType','Text',10)
    arcpy.management.AddField(IntAppOverlay,'Major','Short')
    uc = arcpy.UpdateCursor(IntAppOverlay)
    for row in uc:
        int_id = row.getValue('Int_ID')
        obj_id = row.getValue('OBJECTID')
        row.setValue('IFType',df1.loc[int_id,'IFType'])
        row.setValue('Major',int(df.loc[obj_id,'Major']))
        uc.updateRow(row)   
    del uc
    del row
    print(strftime("%Y-%m-%d %H:%M:%S"))
    
    Diss = hsmpy3.common.CreateOutPath(MainFile=Project_GDB+'\\Int_App_Att_Diss',Extension='',appendix=str(year))
    arcpy.DissolveRouteEvents_lr (in_events = IntAppOverlay, 
                                  in_event_properties = ' '.join(['RID','LINE','BMP','EMP']), 
                                  dissolve_field = ['Int_ID','IFType','Major'], 
                                  out_table = Diss, 
                                  out_event_properties = ' '.join(['RID','LINE','BMP','EMP']), 
                                  dissolve_type = 'DISSOLVE', 
                                  build_index = 'INDEX')
    print(strftime("%Y-%m-%d %H:%M:%S"))
    
    SegIntAtt = hsmpy3.common.CreateOutPath(MainFile=Project_GDB+'\\SegAtt_Int',appendix=str(year),Extension='')
    arcpy.lr.OverlayRouteEvents(
            in_table                 = Project_GDB+'\\SegAtt_' + str(year),
            in_event_properties      = ' '.join(['ROUTE_ID','LINE','BEG_POINT','END_POINT']),
            overlay_table            = Diss,
            overlay_event_properties = ' '.join(['RID','LINE','BMP','EMP']),
            overlay_type             = "UNION", 
            out_table                = SegIntAtt, 
            out_event_properties     = ' '.join(['RID','LINE','BMP','EMP']),
            zero_length_events       = "ZERO", 
            in_fields                = "FIELDS", 
            build_index              = "INDEX"
    )        
    print(year)

2018-09-06 15:36:26
2018-09-06 15:45:27
2018-09-06 16:00:15
2018-09-06 16:03:43
2018-09-06 16:05:07
2012
2018-09-06 16:08:41
2018-09-06 16:17:53
2018-09-06 16:32:33
2018-09-06 16:36:01
2018-09-06 16:37:22
2013
2018-09-06 16:40:49
2018-09-06 16:49:54
2018-09-06 17:05:05
2018-09-06 17:08:36
2018-09-06 17:09:57
2014
2018-09-06 17:13:38
2018-09-06 17:23:09
2018-09-06 17:38:24
2018-09-06 17:41:57
2018-09-06 17:43:18
2015
2018-09-06 17:47:00
2018-09-06 17:56:36
2018-09-06 18:13:44
2018-09-06 18:17:23
2018-09-06 18:18:43
2016
